In [1]:
from concurrent.futures import ThreadPoolExecutor
import json
import os
from openai import OpenAI
import logging
from time import perf_counter, sleep
import re
from threading import RLock
from dotenv import load_dotenv

load_dotenv()

log_format = '[ %(levelname)s] [%(asctime)s] [%(module)s] [%(lineno)s] [%(message)s]'
logging.basicConfig(level=logging.DEBUG, format=log_format)

file_handler = logging.FileHandler('logfile.log')
file_handler.setLevel(logging.DEBUG)

formatter = logging.Formatter(log_format)
file_handler.setFormatter(formatter)

log = logging.getLogger()
log.addHandler(file_handler)

lock = RLock()
converted_qno = []
failed_qno = []
reponse_data = []



[ INFO] [2024-04-29 15:46:50,340] [openai] [23] [model: gpt-3.5-turbo]


In [2]:

SYSTEM_PROMPT = """
**You are a UPSC prelims question expert specializing in converting English MCQs to Hindi.**

**Given a question in English (statement) and a hint, convert it to Hindi (statement) suitable for the UPSC prelims exam, maintaining technical accuracy and UPSC context.**

**Input (JSON):**

* `statement`: Question statement with options in English.
* `hint`: Hint for the question in English.

**Output (JSON):**

* `statement`: Converted question statement with options in Hindi.
* `hint`: Hint for the question in Hindi.

**IMPORTANT INFORMATION**
- DONOT change the base statement format. The number of options should remain the same in the converted statement.
- DONOT forget to translate choices from end of each input `statement` to reponse `statement`.
"""

In [3]:
def clean_text(text, clean=True):
    if not clean:
        return re.sub(r'\s+', ' ', text.strip())
        # return text.strip()
    return re.sub(r'\s+', ' ', text.replace("\\n", " ").replace("", " ").strip())


In [4]:
def get_data():
    input_data = []
    input_data_dict = {}
    qno = []
    with open('data.json', 'r') as fp:
        json_data = json.load(fp)
    for key, item in json_data.items():
        statement = clean_text(item['statement'])
        hint = clean_text(item['hint'])
        qno = item["Qno"]
        temp = {'statement': statement, 'hint': hint, "Qno": qno}
        input_data.append(temp)
        input_data_dict[key] = temp
    # save_input_data(input_data_dict)
    return input_data


In [5]:
def convert_question(input_data, OPENAI_KEY):
    qno = input_data.pop("Qno")
    try:
        USER_PROMPT = str(input_data)
        data = OpenAI.generate_completion(SYSTEM_PROMPT, USER_PROMPT, OPENAI_KEY)
        if not data:
            raise Exception('Opena ai didnot respond')
        data["Qno"] = qno
        with lock:
            converted_qno.append(qno)
        reponse_data.append(data)
        # with lock:
            # save_data(reponse_data)

    except Exception as e:
        log.error(str(e))
        # with lock:
        failed_qno.append(qno)

In [6]:
def get_converted_questions():
    question_path = 'converted/questions_1.json'
    if os.path.exists(question_path):
        with open(question_path, 'r') as fp:
            file_data = json.load(fp)
        return [data['Qno'] for data in file_data]
    return []

In [7]:
def save_data():
    question_path = 'converted/questions_1.json'
    file_data = []
    try:
        if os.path.exists(question_path):
            with open(question_path, 'r') as fp:
                file_data = json.load(fp)

            with open(f'converted/questions_1_copy.json', 'w') as fp:
                json.dump(file_data, fp, ensure_ascii=False)

        file_data.extend(reponse_data)
        with open(question_path, 'w') as fp:
            json.dump(file_data, fp, ensure_ascii=False)
        return True

    except Exception as e:
        log.error(str(e))
        return None

In [9]:
input_data = get_data()
converted_qno.extend(get_converted_questions())
OPENAI_KEY_1 = os.getenv('OPENAI_KEY_1')

log.info(converted_qno)
log.info(len(converted_qno))

[ INFO] [2024-04-29 15:46:57,641] [2143225324] [5] [[0, 1, 10, 100, 1000, 1001, 1003, 1004, 1005, 1006, 1008, 1009, 101, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 102, 1020, 1021, 1022, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1023, 1007, 1023, 1024, 1007, 1023, 1024, 1025, 1007, 1023, 1024, 1025, 1026, 1007, 1023, 1024, 1025, 1026, 1027, 1007, 1023, 1024, 1025, 1026, 1027, 1028, 1007, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 0, 1, 10, 100, 1000, 1001, 1003, 1004, 1005, 1006, 1008, 1009, 101, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 102, 1020, 1021, 1022, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1007, 1023, 1007, 1023, 1024, 1007, 1023, 1024, 1025, 1007, 1023, 1024, 1025, 1026, 1007, 1023, 1024, 1025, 1026, 1027, 1007, 1023, 1024, 1025, 1026, 1027, 1028, 1007, 1023, 1024, 1025, 1026, 1027, 1028, 1029]]
[ INFO] 

In [44]:
for data in input_data:
    qno = data["Qno"]
    if qno in converted_qno:
        log.debug(f'{qno} already converted')
    log.info(f'Converting question {qno}')
    try:
        convert_question(data, OPENAI_KEY=OPENAI_KEY_1)
        save_data()
        log.info(f'Converted question {qno}')
    except Exception as e:
        log.error(str(e))


[ INFO] [2024-04-29 15:28:09,188] [3818965748] [3] [Converting question 0]
[ DEBUG] [2024-04-29 15:28:09,189] [2206373033] [4] [0 already converted]
[ INFO] [2024-04-29 15:28:09,192] [3818965748] [7] [Converted question 0]
[ INFO] [2024-04-29 15:28:09,193] [3818965748] [3] [Converting question 1]
[ DEBUG] [2024-04-29 15:28:09,194] [2206373033] [4] [1 already converted]
[ INFO] [2024-04-29 15:28:09,197] [3818965748] [7] [Converted question 1]
[ INFO] [2024-04-29 15:28:09,198] [3818965748] [3] [Converting question 10]
[ DEBUG] [2024-04-29 15:28:09,198] [2206373033] [4] [10 already converted]
[ INFO] [2024-04-29 15:28:09,201] [3818965748] [7] [Converted question 10]
[ INFO] [2024-04-29 15:28:09,202] [3818965748] [3] [Converting question 100]
[ DEBUG] [2024-04-29 15:28:09,203] [2206373033] [4] [100 already converted]
[ INFO] [2024-04-29 15:28:09,206] [3818965748] [7] [Converted question 100]
[ INFO] [2024-04-29 15:28:09,207] [3818965748] [3] [Converting question 1000]
[ DEBUG] [2024-04-29 

KeyboardInterrupt: 

In [45]:
reponse_data

[{'statement': 'निम्नलिखित में से कौन एक मैक्रो-आर्थिक स्थिरीकरण नीति नहीं है? (ए) गरीबी निवारण के लिए उच्च सरकारी व्यय। (ब) एफआरबीएम अधिनियम (सी) मुद्रास्फीति लक्ष्य (डी) निर्यात प्रोत्साहन ऑफर्स',
  'hint': 'मैक्रो-आर्थिक एक शाखा है जो अर्थशास्त्र क्षेत्र का है जो संग्रहीय अर्थव्यवहार का अध्ययन करती है। इसमें एक अर्थव्यवहार और प्रदर्शन का अध्ययन किया जाता है जैसे कि, मुद्रास्फीति, मूल्य स्तर, वृद्धि दर, राष्ट्रीय आय, कुल घरेलू उत्पाद और बेरोजगारी में परिवर्तन। मैक्रो-आर्थिक स्थिरता का मतलब नियंत्रित वित्तीय घाटा, नियंत्रित चालन खाता घाटा, और स्थिर वृद्धि के साथ नियंत्रित मुद्रास्फीति है। गैर पूंजीगत व्यय के लिए उच्च सरकारी व्यय स्थिरीकरण की संभावना को बढ़ावा देगा। वित्तीय जिम्मेदारी और बजट प्रबंधन अधिनियम, 2003 (एफआरबीएमए) भारतीय संसद का एक अधिनियम है जो वित्तीय अनुशासन को संस्थागत करने, भारत के वित्तीय घाटे को कम करने, मैक्रो-आर्थिक प्रबंधन को सुधारने और सार्वजनिक निधियों के सम्पूर्ण प्रबंधन को संतुलित बजट की ओर ले जाने के लिए है। मुद्रास्फीति लक्ष्यित करना एक मौद्रिक नीति शास्त्र ह

In [14]:
converted_qno

[]